In [1]:
import cv2
import datetime
import numpy as np 
import os
import pandas as pd
import skimage.io as io
import skimage.transform as trans
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers, models
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, TensorBoard
from tensorflow.keras import backend as keras
import tensorflow as tf

In [332]:
X_test = np.load('etpad_test_data_1_nwdel.npy')
y_test = np.load('etpad_test_labels_1_nwdel.npy')

In [335]:
np.min(X_test)

-1.992440511965718

In [3]:
def conv_block(x, filter_size, size, stride, batch_norm=False, multiple=False):
    
    conv = layers.Conv1D(size, filter_size, strides = stride, padding="same", kernel_initializer = 'he_normal')(x)
    if multiple:
        conv = layers.Conv1D(size, filter_size, strides = stride, padding="same", kernel_initializer = 'he_normal')(x)
        conv = layers.Conv1D(size, filter_size, strides = stride, padding="same", kernel_initializer = 'he_normal')(x)
            
    if batch_norm is True:
        conv = layers.BatchNormalization()(conv)
    conv = layers.Activation("relu")(conv)
    
    return conv

In [32]:
def oleg_model(input_shape, NUM_CLASSES=1, dropout_rate=0.0, kernel_size = 3, batch_norm=True, final_activation = 'linear'):
    inputs1 = layers.Input(input_shape, dtype=tf.float32)
    
    block_1 = conv_block(inputs1, kernel_size, 64, 1, batch_norm=True)
    block_2 = conv_block(block_1, kernel_size, 64, 1, batch_norm=True, multiple=True)
    block_3 = conv_block(block_2, kernel_size, 128, 1, batch_norm=True)
    block_4 = conv_block(block_3, kernel_size, 128, 1, batch_norm=True, multiple=True)
    block_5 = conv_block(block_4, kernel_size, 256, 1, batch_norm=True)
    block_6 = conv_block(block_5, kernel_size, 256, 1, batch_norm=True, multiple=True)
    block_7 = conv_block(block_6, kernel_size, 512, 1, batch_norm=True)
    block_8 = conv_block(block_7, kernel_size, 512, 1, batch_norm=True, multiple=True)
    
    average_pooling = AveragePooling1D(pool_size=2, strides=2, padding='same')(block_8)
    flatten = tf.reshape(average_pooling,[-1])
    oleg_final = layers.Dense(1, activation = final_activation)(average_pooling)
    
    model = models.Model(inputs1, oleg_final, name="Oleg_Model")
    return model

In [5]:
def multihead_oleg_model(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):
    inputs1 = layers.Input(input_shape, dtype=tf.float32)
    
    block_1_a = conv_block(inputs1, 3, 64, 1, batch_norm=True)
    block_2_a = conv_block(block_1_a, 3, 64, 1, batch_norm=True, multiple=True)
    block_3_a = conv_block(block_2_a, 3, 128, 1, batch_norm=True)
    block_4_a = conv_block(block_3_a, 3, 128, 1, batch_norm=True, multiple=True)
    average_pooling_a = AveragePooling1D(pool_size=2, strides=2, padding='same')(block_4_a)
    flatten_a = layers.Flatten()(average_pooling_a)
    
    block_1_b = conv_block(inputs1, 5, 64, 1, batch_norm=True)
    block_2_b = conv_block(block_1_b, 5, 64, 1, batch_norm=True, multiple=True)
    block_3_b = conv_block(block_2_b, 5, 128, 1, batch_norm=True)
    block_4_b = conv_block(block_3_b, 5, 128, 1, batch_norm=True, multiple=True)
    average_pooling_b = AveragePooling1D(pool_size=2, strides=2, padding='same')(block_4_b)
    flatten_b = layers.Flatten()(average_pooling_b)
        
    block_1_c = conv_block(inputs1, 7, 64, 1, batch_norm=True)
    block_2_c = conv_block(block_1_c, 7, 64, 1, batch_norm=True, multiple=True)
    block_3_c = conv_block(block_2_c, 7, 128, 1, batch_norm=True)
    block_4_c = conv_block(block_3_c, 7, 128, 1, batch_norm=True, multiple=True)
    average_pooling_c = AveragePooling1D(pool_size=2, strides=2, padding='same')(block_4_c)
    flatten_c = layers.Flatten()(average_pooling_c)
    
    flatten = tf.keras.layers.concatenate([flatten_a, flatten_b, flatten_c])
    oleg_final = layers.Dense(100, activation = "relu")(flatten)
    oleg_final = layers.Dense(1, activation = "sigmoid")(oleg_final)
    
    model = models.Model(inputs1, oleg_final, name="Multihead_Model")
    return model

In [6]:
model_m = multihead_oleg_model(input_shape = (2,1500))
model_m.summary()

Model: "Multihead_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2, 1500)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 2, 64)        288064      ['input_1[0][0]']                
                                                                                                  
 conv1d_8 (Conv1D)              (None, 2, 64)        480064      ['input_1[0][0]']                
                                                                                                  
 conv1d_16 (Conv1D)             (None, 2, 64)        672064      ['input_1[0][0]']                
                                                                                    

 activation_3 (Activation)      (None, 2, 128)       0           ['batch_normalization_3[0][0]']  
                                                                                                  
 activation_7 (Activation)      (None, 2, 128)       0           ['batch_normalization_7[0][0]']  
                                                                                                  
 activation_11 (Activation)     (None, 2, 128)       0           ['batch_normalization_11[0][0]'] 
                                                                                                  
 average_pooling1d (AveragePool  (None, 1, 128)      0           ['activation_3[0][0]']           
 ing1D)                                                                                           
                                                                                                  
 average_pooling1d_1 (AveragePo  (None, 1, 128)      0           ['activation_7[0][0]']           
 oling1D) 

In [7]:
import glob

In [336]:
all_oleg_models = []
for name in glob.glob('D:/ETPAD.v2/models/*'):
    if 'multi' not in name:
        all_oleg_models = all_oleg_models + [name]
        
all_oleg_models

['D:/ETPAD.v2/models\\OLEG_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=128_attack_scenario = 1_activationfunction = linear.hdf5',
 'D:/ETPAD.v2/models\\OLEG_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=128_attack_scenario = 1_activationfunction = sigmoid.hdf5',
 'D:/ETPAD.v2/models\\OLEG_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=16_attack_scenario = 1_activationfunction = linear.hdf5',
 'D:/ETPAD.v2/models\\OLEG_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=16_attack_scenario = 1_activationfunction = sigmoid.hdf5',
 'D:/ETPAD.v2/models\\OLEG_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=64_attack_scenario = 1_activationfunction = linear.hdf5',
 'D:/ETPAD.v2/models\\OLEG_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=64_attack_scenario = 1_activationfunction = sigmoid.hdf5',
 'D:/ETPAD.v2/models\\OLEG_model_kernel_size=3_dropout=0_lr=0.0003_batch_size=128_attack_scenario = 1_activationfunction = linear.hdf5',
 'D:/ETPAD.v2/models\\OLEG_model_kernel_si

In [337]:
results = pd.DataFrame(columns = {'model_name','pred_range','confusion_matrix', 'acr', 'apcer', 'npcer', 'acer'})

In [338]:
for i in range(0,len(all_oleg_models)):
    if i%2==1:
        parameters = all_oleg_models[i].split('\\')[1].split("_")
        k = int(parameters[3].split('=')[1])
        lr = float(parameters[5].split('=')[1])
        activation = parameters[-1].split('=')[1][1:-5]
        
        model_o = oleg_model(input_shape = (2,1500), kernel_size = k, final_activation = activation)
        model_o.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss=tf.keras.losses.binary_crossentropy, 
                metrics='accuracy')
        model_o.load_weights(all_oleg_models[i])
        
        pred = model_o.predict(X_test)
        pred = pred.reshape([pred.shape[0],])
        pred_range_oleg = [np.quantile(pred,[0.,0.25,0.5,0.75,1])]
        
        pred[pred >= 0.5] = 1
        pred[pred<0.5] = 0 
        tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=pred).ravel()
    
    
        apcer = fp/(tn+fp)
        npcer = fn/(fn+tp)
        acr = 1-(0.5*(apcer + npcer))
        acer = 1-acr

        
        results = results.append({'model_name' :all_oleg_models[i],
                                  'pred_range': pred_range_oleg,
                                  'confusion_matrix':[(tn, fp, fn, tp)], 
                                  'acr':acr, 'apcer':apcer, 'npcer':npcer, 'acer':acer }, ignore_index=True)
        

216/216 [==============================] - 5s 23ms/step


C:\Users\anark\AppData\Local\Temp\ipykernel_14720\893954900.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'model_name' :all_oleg_models[i],


216/216 [==============================] - 5s 24ms/step


C:\Users\anark\AppData\Local\Temp\ipykernel_14720\893954900.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'model_name' :all_oleg_models[i],


216/216 [==============================] - 5s 2ms/step


...

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\893954900.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'model_name' :all_oleg_models[i],


216/216 [==============================] - 1s 2ms/step


C:\Users\anark\AppData\Local\Temp\ipykernel_14720\893954900.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'model_name' :all_oleg_models[i],


216/216 [==============================] - 1s 2ms/step


C:\Users\anark\AppData\Local\Temp\ipykernel_14720\893954900.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'model_name' :all_oleg_models[i],


In [339]:
results = results[['model_name', 'acr', 'apcer', 'npcer', 'acer','confusion_matrix','pred_range']]

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\477038574.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  with pd.ExcelWriter('D:/ETPAD.v2/results_unbalanced_attack_1_oleg_model.xls') as writer:


In [143]:
pred = model_o.predict(X_test)
pred = pred.reshape([pred.shape[0],])
np.quantile(pred,[0.,0.25,0.5,0.75,1])

268/268 [==============================] - 1s 3ms/step


array([3.06605700e-14, 4.43045670e-11, 6.08758728e-06, 9.99944806e-01,
       1.00000000e+00])

In [123]:
pred = model_m.predict(X_test)
pred = pred.reshape([pred.shape[0],])
np.quantile(pred,[0.,0.25,0.5,0.75,1])

268/268 [==============================] - 1s 3ms/step


array([2.57429820e-05, 4.81818855e-01, 4.95703965e-01, 4.98012424e-01,
       5.46081007e-01])

In [144]:
pred[pred >= 0.5] = 1
pred[pred<0.5] = 0 
tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=pred).ravel()
tn, fp, fn, tp

(4865, 105, 559, 3016)

In [145]:
apcer = fp/(tn+fp)
npcer = fn/(fn+tp)

acr = 1-(0.5*(apcer + npcer))

acer = 1-acr

In [146]:
acr, apcer, npcer, acer

(0.9112548015364916,
 0.02112676056338028,
 0.15636363636363637,
 0.08874519846350837)

In [147]:
confusion_matrix(y_true=y_test, y_pred=pred)

array([[4865,  105],
       [ 559, 3016]], dtype=int64)

In [103]:
np.quantile(pred,[0.,0.25,0.5,0.75,1])

array([2.57429820e-05, 4.81818855e-01, 4.95703965e-01, 4.98012424e-01,
       5.46081007e-01])

In [341]:
multi_model = []
for name in glob.glob('D:/ETPAD.v2/models/*'):
    if 'multi' in name:
        multi_model = multi_model + [name]
multi_model

['D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=128_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=16_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=64_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0003_batch_size=128_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0003_batch_size=16_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0003_batch_size=64_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.001_batch_size=128_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/mode

In [211]:
i = 2

parameters = multi_model[i].split('\\')[1].split("_")

k = parameters[4]
k = int(k.split('=')[1])

lr = parameters[6]
lr = float(lr.split('=')[1])

model_m = multihead_oleg_model(input_shape = (2,1500))
model_m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss=tf.keras.losses.binary_crossentropy, 
                metrics='accuracy')

model_m.load_weights(multi_model[i])
pred = model_m.predict(X_test)


268/268 [==============================] - 1s 3ms/step


In [ ]:
results = pd.DataFrame(columns = {'model_name','pred_range','confusion_matrix', 'acr', 'apcer', 'npcer', 'acer'})

In [271]:
multi_model[-6].split('\\')[1].split("_")

['OLEG',
 'multi',
 'model',
 'kernel',
 'size=3',
 'dropout=0',
 'lr=0.001',
 'batch',
 'size=128',
 'attack',
 'scenario = 2',
 'activationfunction = None.hdf5']

In [15]:
oleg_model[-6].split('\\')[1].split("_")

['OLEG',
 'model',
 'kernel',
 'size=7',
 'dropout=0',
 'lr=0.01',
 'batch',
 'size=128',
 'attack',
 'scenario = 1',
 'activationfunction = linear.hdf5']

In [270]:
parameters

['OLEG',
 'model',
 'kernel',
 'size=3',
 'dropout=0',
 'lr=0.0001',
 'batch',
 'size=64',
 'attack',
 'scenario = 1.hdf5']

In [19]:
X_test = np.load('etpad_test_data_1.npy')
y_test = np.load('etpad_test_labels_1.npy')

In [21]:
model_m.load_weights(multi_model[0])

In [23]:
multi_model[0]

'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=128_attack_scenario = 1.hdf5'

In [156]:


batches = parameters[8]
batches = int(batches.split('=')[1])



268/268 [==============================] - 1s 2ms/step


In [165]:
pred = model_m.predict(X_test)
pred = pred.reshape([pred.shape[0],])

268/268 [==============================] - 1s 3ms/step


In [158]:
np.quantile(pred,[0.,0.25,0.5,0.75,1])

array([5.30124729e-11, 9.55505399e-08, 1.63642449e-06, 9.91326392e-01,
       1.00000000e+00])

In [167]:
np.quantile(pred,[0.60])

array([0.0001399])

In [159]:
pred[pred >= 0.5] = 1
pred[pred<0.5] = 0 

In [160]:
tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=pred).ravel()

In [161]:
tn, fp, fn, tp

(4912, 58, 971, 2604)

In [162]:
tn, fp, fn, tp

(4912, 58, 971, 2604)

In [163]:
apcer = fp/(tn+fp)
npcer = fn/(fn+tp)

acr = 1-(0.5*(apcer + npcer))

acer = 1-acr

In [164]:
acr, apcer, npcer, acer

(0.8583607941354421,
 0.011670020120724347,
 0.2716083916083916,
 0.14163920586455792)

In [264]:
confusion_matrix(y_true=y_test, y_pred=pred)

array([[4869,  101],
       [ 544, 3031]], dtype=int64)

In [119]:
y_test.shape[0] - np.sum(y_test)

4970

In [120]:
4899+71

4970

In [121]:
733+2842

3575

In [216]:
parameters = multi_model[9].split('\\')[1].split("_")
parameters

['OLEG',
 'multi',
 'model',
 'kernel',
 'size=3',
 'dropout=0',
 'lr=0.01',
 'batch',
 'size=128',
 'attack',
 'scenario = 1',
 'activationfunction = None.hdf5']

In [342]:
multi_model = []
for name in glob.glob('D:/ETPAD.v2/models/*'):
    if 'multi' in name:
        multi_model = multi_model + [name]
multi_model

['D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=128_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=16_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0001_batch_size=64_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0003_batch_size=128_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0003_batch_size=16_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.0003_batch_size=64_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/models\\OLEG_multi_model_kernel_size=3_dropout=0_lr=0.001_batch_size=128_attack_scenario = 1_activationfunction = None.hdf5',
 'D:/ETPAD.v2/mode

In [343]:
results1 = pd.DataFrame(columns = {'model_name','pred_range','confusion_matrix', 'acr', 'apcer', 'npcer', 'acer'})

In [344]:
for i in range(0, len(multi_model)):
    parameters = multi_model[i].split('\\')[1].split("_")

    lr = parameters[6]
    lr = float(lr.split('=')[1])
    print(lr)

    model_m = multihead_oleg_model(input_shape = (2,1500))
    model_m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss=tf.keras.losses.binary_crossentropy, 
                    metrics='accuracy')
    model_m.load_weights(multi_model[i])

    pred = model_m.predict(X_test)
    pred = pred.reshape([pred.shape[0],])
    pred_range_multi = np.quantile(pred,[0.,0.25,0.5,0.75,1])
    
    pred[pred >= 0.5] = 1
    pred[pred<0.5] = 0 
    tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=pred).ravel()
    print([(tn, fp, fn, tp)])

    apcer = fp/(tn+fp)
    npcer = fn/(fn+tp)
    acr = 1-(0.5*(apcer + npcer))
    acer = 1-acr
    
    results1 = results1.append({'model_name' :multi_model[i],
                                'pred_range': pred_range_multi,
                                'confusion_matrix':[(tn, fp, fn, tp)], 
                                'acr':acr, 'apcer':apcer, 'npcer':npcer, 'acer':acer }, ignore_index=True)
    

0.0001
216/216 [==============================] - 1s 4ms/step
[(4639, 183, 261, 1814)]
0.0001


C:\Users\anark\AppData\Local\Temp\ipykernel_14720\181655120.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results1 = results1.append({'model_name' :multi_model[i],


216/216 [==============================] - 1s 3ms/step
[(4739, 83, 459, 1616)]
0.0001


C:\Users\anark\AppData\Local\Temp\ipykernel_14720\181655120.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results1 = results1.append({'model_name' :multi_model[i],


216/216 [==============================] - 1s 3ms/step
[(4683, 139, 328, 1747)]
0.0003


...

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\181655120.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results1 = results1.append({'model_name' :multi_model[i],


216/216 [==============================] - 1s 2ms/step
[(4719, 103, 383, 1692)]
0.01


C:\Users\anark\AppData\Local\Temp\ipykernel_14720\181655120.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results1 = results1.append({'model_name' :multi_model[i],


216/216 [==============================] - 1s 2ms/step
[(4704, 118, 293, 1782)]


C:\Users\anark\AppData\Local\Temp\ipykernel_14720\181655120.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results1 = results1.append({'model_name' :multi_model[i],


In [345]:
results1 = results1[['model_name', 'acr', 'apcer', 'npcer', 'acer','confusion_matrix','pred_range']]
results = results[['model_name', 'acr', 'apcer', 'npcer', 'acer','confusion_matrix','pred_range']]
with pd.ExcelWriter('D:/ETPAD.v2/results_clipped_attack_1_oleg_model.xls') as writer:
    results.to_excel(writer, sheet_name='Oleg_Model')
    results1.to_excel(writer, sheet_name='Multihead_Model')

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\2122465480.py:3: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  with pd.ExcelWriter('D:/ETPAD.v2/results_clipped_attack_1_oleg_model.xls') as writer:


In [360]:
pred = model_o.predict(X_test)
pred = pred.reshape([pred.shape[0],])
pred[pred > -0.5] = 1
pred[pred<=-0.5] =  0

268/268 [==============================] - 1s 2ms/step


In [361]:
tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=pred).ravel()

In [362]:
apcer = fp/(tn+fp)
npcer = fn/(fn+tp)

acr = 1-(0.5*(apcer + npcer))

acer = 1-acr

In [363]:
acr, apcer, npcer, acer

(0.487792911314038,
 0.9997987927565393,
 0.024615384615384615,
 0.512207088685962)

In [364]:
confusion_matrix(y_true=y_test, y_pred=pred).ravel()

array([   1, 4969,   88, 3487], dtype=int64)

In [365]:
tn, fp, fn, tp

(1, 4969, 88, 3487)

In [294]:
i = 5
parameters = multi_model[i].split('\\')[1].split("_")

lr = parameters[6]
lr = float(lr.split('=')[1])
print(lr)

model_m = multihead_oleg_model(input_shape = (2,1500))
model_m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss=tf.keras.losses.binary_crossentropy, 
                metrics='accuracy')
model_m.load_weights(multi_model[i])

pred = model_m.predict(X_test)
pred = pred.reshape([pred.shape[0],])
pred_range_multi = np.quantile(pred,[0.,0.25,0.5,0.75,1])

pred[pred >= 0.5] = 1
pred[pred<0.5] = 0 
tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=pred).ravel()
print([(tn, fp, fn, tp)])

apcer = fp/(tn+fp)
npcer = fn/(fn+tp)
acr = 1-(0.5*(apcer + npcer))
acer = 1-acr

0.0003
317/317 [==============================] - 1s 3ms/step
[(4939, 36, 4769, 386)]


In [295]:
np.shape(y_test)[0]-np.sum(y_test)

4975.0

In [309]:
pred = model_m.predict(X_test)
pred = pred.reshape([pred.shape[0],])
np.quantile(pred,[0.,0.25,0.5,0.75,1])

317/317 [==============================] - 1s 3ms/step


array([2.98655188e-16, 3.86469942e-15, 2.52869023e-14, 9.06734255e-13,
       1.00000000e+00])

In [301]:
sample_result = pd.DataFrame(columns = {'True', 'Predicted'})  

In [302]:
y_test.shape, pred.shape

((10130, 1), (10130,))

In [303]:
pred

array([5.2229945e-14, 5.0107407e-15, 2.4167243e-13, ..., 2.0464886e-15,
       1.0014779e-14, 2.6315509e-11], dtype=float32)

In [304]:
for i in range(0,10130):
    sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.ap

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.ap

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.ap

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.ap

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.ap

...

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.ap

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.ap

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.ap

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.ap

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_result = sample_result.append({'True': y_test[i], 'Predicted': pred[i]},  ignore_index=True)
C:\Users\anark\AppData\Local\Temp\ipykernel_14720\3329830642.py:2: FutureWarning: The frame.ap

In [305]:
sample_result_result.head()

,True,Predicted
0,[0.0],0.0
1,[1.0],0.0
2,[0.0],0.0
3,[0.0],0.0
4,[0.0],0.0


In [306]:
sample_result.to_excel('D:/ETPAD.v2/sample_result_att2.xls')

C:\Users\anark\AppData\Local\Temp\ipykernel_14720\720563040.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  sample_result.to_excel('D:/ETPAD.v2/sample_result_att2.xls')


In [322]:
pred = model_m.predict(X_test)
pred = pred.reshape([pred.shape[0],])
pred[pred >= 0.00000000000001] = 1
pred[pred<0.00000000000001] = 0 
tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=pred).ravel()

317/317 [==============================] - 1s 3ms/step


In [323]:
tn, fp, fn, tp

(3217, 1758, 749, 4406)

In [325]:
tn, fp, fn, tp

(3217, 1758, 749, 4406)

In [326]:
apcer = fp/(tn+fp)
npcer = fn/(fn+tp)

acr = 1-(0.5*(apcer + npcer))

acer = 1-acr

In [327]:
acr

0.7506686682685981

In [328]:
 confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()

array([0, 2, 1, 1], dtype=int64)

#### LSTM models

In [2]:
def lstm_block(x, size, multiple=False):
    
    conv = layers.LSTM(size, activation = 'tanh', input_shape=(2,1500), kernel_initializer = 'he_normal',  return_sequences=True)(x)
    if multiple:
        conv = layers.LSTM(size, activation = 'tanh', input_shape=(2,1500), kernel_initializer = 'he_normal',return_sequences=True)(x)
        conv = layers.LSTM(size, activation = 'tanh', input_shape=(2,1500), kernel_initializer = 'he_normal',return_sequences=True)(x)
    return conv

def lstm_model(input_shape, NUM_CLASSES=1, dropout_rate=0.0):
    inputs1 = layers.Input(input_shape, dtype=tf.float32)
    block_1 = lstm_block(inputs1,  64)
    block_1 = Dropout(0.35)(block_1, training=True)
    
    block_2 = lstm_block(block_1,  128)
    block_2 = Dropout(0.35)(block_2, training=True)
    
    block_3 = lstm_block(block_2,  256)
    block_3 = Dropout(0.35)(block_3, training=True)
    
    block_4 = lstm_block(block_3,  512, multiple=True)
    block_4 = Dropout(0.35)(block_4, training=True)
    
    dense_1 = layers.Dense(1, activation = 'sigmoid')(block_4)
    model = models.Model(inputs1, dense_1, name="LSTM_model")
    return model
    

In [8]:
X_test = np.load('test_eptad_data_nw_del_2_sas1.npy')
y_test = np.load('test_eptad_label_nw_del_2_sas1.npy')
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [9]:
X_test = np.swapaxes(X_test, 1,2)

In [11]:
model = lstm_model(input_shape = (1500,2))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.binary_crossentropy, 
                metrics='accuracy')
model.load_weights(r"D:\ETPAD.v2\models_lstm_1\LSTM_model_kernel_size=3_dropout=0_lr=0.001_batch_size=64_attack_scenario = 1_activationfunction = sigmoid.hdf5")

In [12]:
pred = model.predict(X_test)

216/216 [==============================] - 32s 129ms/step


In [14]:
pred.shape

(6897, 1500, 1)

In [18]:
pred = pred[:,-1,:]
pred = pred.reshape([pred.shape[0],])
pred_range_multi = np.quantile(pred,[0.,0.25,0.5,0.75,1])

pred[pred >= 0.5] = 1
pred[pred<0.5] = 0 
tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=pred).ravel()
print([(tn, fp, fn, tp)])

apcer = fp/(tn+fp)
npcer = fn/(fn+tp)
acr = 1-(0.5*(apcer + npcer))
acer = 1-acr

[(4821, 1, 1, 2074)]


In [20]:
acr,apcer,npcer,acer

(0.9996553447302274,
 0.00020738282870178348,
 0.00048192771084337347,
 0.00034465526977256467)

In [21]:
print([(tn, fp, fn, tp)])

[(4821, 1, 1, 2074)]
